In [ ]:
# Install evaluate library for load_metric function
%pip install evaluate

In [ ]:
%pip install --upgrade accelerate

Run on ANLI-2k


In [ ]:
"google/electra-small-discriminator"
"FacebookAI/roberta-base"
"google-bert/bert-base-uncased"

Train on full to get training dynamics

In [ ]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path "google/electra-small-discriminator" \
    --output_dir ./data-model/anli-2k/all-data \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy

^C


Save feature embeddings and importance scores

In [ ]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/anli-2k/all-data \
    --do_eval \
    --eval_train \
    --save_feature \
    --save_confidence \
    --save_importance_scores \
    --training_dynamics \
    --per_device_eval_batch_size 16 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy

Prun and train on 50% data

In [ ]:
CORESET_RATIO = 0.5
N_NEIGHBOR = 10
GAMMA = 0.1

%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path roberta-base \
    --output_dir ./data-model/anli-2k/coreset-0.5 \
    --do_train \
    --do_test \
    --coreset \
    --coreset-mode class \
    --budget-mode uniform \
    --sampling-mode graph \
    --data-score-path ./data-model/anli-2k/all-data/data-score-goddawg.pickle \
    --feature-path ./data-model/anli-2k/all-data/train-features.npy \
    --coreset-key forgetting \
    --coreset-ratio {CORESET_RATIO} \
    --mis-ratio 0.4 \
    --label-balanced \
    --num_train_epochs 10 \
    --per_device_train_batch_size 32 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy \
    --n-neighbor {N_NEIGHBOR} \
    --gamma {GAMMA} \
    --graph-mode sum \
    --graph-sampling-mode weighted

Train D(Full data) and D'(50% coreset) on BERT base uncased

In [ ]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path bert-base-uncased \
    --output_dir ./data-model/anli-2k/roberta_base/bert/ \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy

In [ ]:
%cd d2pruning/
!python train_nlp_explore.py \
    --task_name "goddawg/anli-2k" \
    --model_name_or_path bert-base-uncased \
    --output_dir ./data-model/anli-2k/roberta_base/bert-50%/ \
    --do_train \
    --do_test \
    --train_logger \
    --num_train_epochs 10 \
    --per_device_train_batch_size 16 \
    --learning_rate 2e-5 \
    --max_length 256 \
    --val-index-path ./data-model/anli-2k/val_index.npy \
    --train-index-path ./data-model/anli-2k/roberta_base/coreset-50%/coreset-goddawg.npy